In [6]:
#Setting up libraries and importing data

import pandas as pd 
import numpy as np

loanbook = pd.read_csv("loanbook.csv", sep = "|")
cpr = pd.read_csv("cpr.csv", sep = "|")
erc_lookup = pd.read_csv("erc.csv", sep = "|")

In [7]:
loanbook.head()

,loan_id,product,rate_term,origination_date,reversion_date,loan_amount,initial_rate,reversion_rate,term,interest_only_amount,upfront_fees,upfront_costs,adjust Nov-21
0,0,Product A,19,2019-11-10,2021-06-10,4823742.87,0.12,0.20,118,0,6000,3400,400
1,1,Product B,24,2021-02-17,2023-02-17,67850.94,0.15,0.18,76,0,3000,3000,1000
2,2,Product C,37,2019-11-24,2022-12-24,4470382.34,0.11,0.18,79,0,6000,2800,1000
3,3,Product D,44,2021-02-01,2024-10-01,155836.09,0.11,0.14,97,0,6000,3400,600
4,4,Product E,18,2020-02-16,2021-08-16,146592.93,0.10,0.14,120,0,5000,2600,400


In [8]:
cpr.head()

,Product,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,...,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120
0,Product A,1,1,0.974277,0.983138,0.981345,0.960939,0.966870,0.955572,0.958963,0.948155,0.944708,0.940629,0.938380,0.933454,0.941349,0.928888,0.917339,0.921429,0.930810,0.920160,0.917790,0.903163,0.887157,0.887056,0.895329,0.797072,0.712769,0.653740,0.616427,0.613946,0.578729,0.569482,0.548247,0.549159,0.507849,0.517154,0.481148,0.460964,0.474679,...,0.085627,0.056817,0.074223,0.065093,0.057336,0.068002,0.050697,0.055354,0.060470,0.048082,0.052652,0.042570,0.053338,0.046909,0.026434,0.026334,0.030634,0.030875,0.018179,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
1,Product B,1,1,0.965001,0.972277,0.962384,0.966880,0.951127,0.962494,0.940248,0.960986,0.944520,0.956663,0.930311,0.946387,0.914912,0.926540,0.916084,0.905519,0.895052,0.887755,0.899981,0.901476,0.883865,0.877944,0.865835,0.864290,0.858746,0.854505,0.828302,0.817892,0.818686,0.835202,0.796906,0.800801,0.800750,0.786347,0.797801,0.692315,0.619233,...,0.120025,0.135597,0.106972,0.096006,0.102980,0.090812,0.100421,0.084918,0.076092,0.075657,0.067208,0.079228,0.085776,0.052636,0.074864,0.054074,0.052730,0.061549,0.048576,0.060016,0.056333,0.029007,0.031026,0.036736,0.041041,0.032362,0.041591,0.042052,0.024013,0.030366,0.023581,0.020092,0.024031,0.028435,0.029645,0.018311,0.000000,0.000000,0.000000,0.0
2,Product C,1,1,0.990667,0.995642,0.978303,0.987434,0.996079,0.994009,0.966627,0.983410,1.000000,0.978725,0.983681,0.956833,0.960375,0.996198,0.961632,0.984849,0.974309,0.963229,0.956100,0.973861,0.950858,0.967041,0.961184,0.968899,0.962238,0.937619,0.946426,0.943115,0.956479,0.926399,0.940973,0.960181,0.951449,0.925616,0.935806,0.919178,0.926796,...,0.258363,0.264379,0.249030,0.272012,0.244668,0.260035,0.255530,0.261117,0.252458,0.256025,0.245734,0.263940,0.261826,0.258891,0.248507,0.273500,0.266089,0.262017,0.257542,0.241722,0.258131,0.248167,0.255265,0.260049,0.244426,0.265264,0.265982,0.249325,0.266095,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
3,Product D,1,1,0.967297,0.963433,0.971727,0.960517,0.969556,0.956328,0.965504,0.973455,0.962279,0.970308,0.949764,0.948716,0.945909,0.945847,0.935246,0.956218,0.924431,0.936747,0.919903,0.911530,0.911136,0.908233,0.901246,0.912592,0.895979,0.887683,0.893224,0.903302,0.875132,0.867630,0.875300,0.869561,0.884134,0.847475,0.859889,0.866316,0.848554,...,0.306511,0.270158,0.258803,0.265881,0.246222,0.241078,0.208323,0.198399,0.193163,0.183296,0.195431,0.164371,0.180942,0.150964,0.163673,0.131031,0.148003,0.137538,0.126790,0.109386,0.120302,0.120509,0.096515,0.104542,0.103514,0.087640,0.076373,0.091406,0.062376,0.059514,0.060982,0.069007,0.055786,0.052640,0.056147,0.048488,0.062664,0.037634,0.037734,0.0
4,Product E,1,1,0.987182,0.978085,1.000000,0.997603,0.964585,0.985005,0.955024,0.963205,0.970512,0.960446,0.958807,0.976756,0.956716,0.976991,0.968188,0.953909,0.963377,0.920266,0.889097,0.922191,0.910636,0.911995,0.903880,0.897920,0.812003,0.733139,0.655321,0.635591,0.617908,0.621103,0.600995,0.583436,0.557214,0.556287,0.507421,0.515208,0.503791,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0


In [9]:
erc_lookup.head()

,Product,Month 0,Month 1,Month 2,Month 3,Month 4,Month 5,Month 6,Month 7,Month 8,Month 9,Month 10,Month 11,Month 12,Month 13,Month 14,Month 15,Month 16,Month 17,Month 18,Month 19,Month 20,Month 21,Month 22,Month 23,Month 24,Month 25,Month 26,Month 27,Month 28,Month 29,Month 30,Month 31,Month 32,Month 33,Month 34,Month 35,Month 36,Month 37,Month 38,...,Month 81,Month 82,Month 83,Month 84,Month 85,Month 86,Month 87,Month 88,Month 89,Month 90,Month 91,Month 92,Month 93,Month 94,Month 95,Month 96,Month 97,Month 98,Month 99,Month 100,Month 101,Month 102,Month 103,Month 104,Month 105,Month 106,Month 107,Month 108,Month 109,Month 110,Month 111,Month 112,Month 113,Month 114,Month 115,Month 116,Month 117,Month 118,Month 119,Month 120
0,Product A,0.023451,0.022499,0.023840,0.020951,0.027985,0.031261,0.030891,0.029232,0.031486,0.030706,0.029370,0.022566,0.026849,0.026786,0.028297,0.021665,0.022613,0.023366,0.021795,0.029880,0.021920,0.024771,0.025411,0.028799,0.029840,0.028298,0.023902,0.023264,0.025596,0.020910,0.027684,0.022530,0.027823,0.027105,0.026847,0.027687,0.025251,0.021327,0.029125,...,0.029459,0.024563,0.023302,0.028104,0.026623,0.028895,0.029373,0.023472,0.028391,0.028240,0.022377,0.028879,0.028163,0.030331,0.026426,0.022294,0.030877,0.030540,0.026635,0.021951,0.020300,0.026967,0.021053,0.023956,0.028010,0.022196,0.026580,0.031547,0.022786,0.023449,0.023264,0.030246,0.027078,0.023440,0.024636,0.030245,0.029782,0.027942,0.020571,0.021961
1,Product B,0.031411,0.037664,0.035266,0.035257,0.029754,0.028077,0.029840,0.029007,0.025610,0.034641,0.026246,0.030675,0.035450,0.026282,0.033784,0.034032,0.037855,0.032433,0.026973,0.031108,0.037869,0.027983,0.036212,0.028518,0.029636,0.035737,0.032564,0.029226,0.034820,0.035070,0.036680,0.033740,0.025195,0.026297,0.029406,0.035567,0.029634,0.035295,0.025471,...,0.027787,0.036573,0.033327,0.027594,0.030009,0.028246,0.030322,0.035855,0.032956,0.038690,0.032035,0.034175,0.032330,0.036856,0.026029,0.033186,0.032816,0.033039,0.033928,0.037911,0.038725,0.037277,0.038430,0.029781,0.031029,0.026517,0.031982,0.031280,0.034757,0.029624,0.025242,0.029839,0.037438,0.027442,0.027768,0.031506,0.037209,0.025677,0.035159,0.025401
2,Product C,0.020331,0.021578,0.026381,0.022798,0.023268,0.019685,0.029310,0.022133,0.020853,0.022897,0.020822,0.027249,0.024583,0.024954,0.023631,0.019384,0.021383,0.028335,0.020602,0.021056,0.027314,0.020603,0.021386,0.021330,0.019874,0.023593,0.019392,0.021878,0.022772,0.029232,0.022375,0.019100,0.023495,0.026638,0.026071,0.019214,0.027954,0.024783,0.019388,...,0.029382,0.020630,0.020138,0.024560,0.025979,0.028458,0.021961,0.024519,0.028570,0.026702,0.028960,0.026929,0.021023,0.026723,0.025286,0.024962,0.024758,0.026098,0.029211,0.022295,0.020148,0.020764,0.022793,0.026247,0.025987,0.022167,0.024392,0.029064,0.027742,0.028988,0.028119,0.025273,0.021198,0.028104,0.024323,0.026807,0.028925,0.027322,0.022638,0.028440
3,Product D,0.005270,0.006673,0.005311,0.006416,0.006424,0.006863,0.006582,0.006319,0.004708,0.006595,0.005281,0.005316,0.006630,0.006538,0.005925,0.005427,0.005278,0.006812,0.006800,0.006273,0.005895,0.005896,0.005073,0.004950,0.005272,0.006038,0.005582,0.005464,0.006069,0.006530,0.004993,0.006559,0.006754,0.006440,0.006799,0.005891,0.005175,0.006842,0.006976,...,0.006384,0.006288,0.006236,0.005299,0.004992,0.004717,0.006597,0.004697,0.006396,0.005703,0.005789,0.005310,0.006197,0.006934,0.005106,0.006015,0.005408,0.006004,0.005474,0.005884,0.004781,0.004552,0.004706,0.006279,0.005876,0.004970,0.005974,0.006043,0.005996,0.006584,0.005675,0.006023,0.004670,0.005090,0.005491,0.005025,0.005193,0.005633,0.005221,0.006872
4,Product E,0.067553,0.079343,0.078965,0.086411,0.066469,0.073358,0.091546,0.071174,0.093933,0.071741,0.096020,0.069230,0.064124,0.072866,0.093662,0.085044,0.074219,0.072085,0.090201,0.077823,0.079602,0.080475,0.072435,0.069958,0.070644,0.079104,0.091161,0.087184,0.095138,0.069634,0.064900,0.077633,0.076793,0.081739,0.089335,0.095532,0.082980,0.065796,0.0

In [10]:
#calculating amount that is repayed monthly, both before and after reversion date
loanbook['monthly_repay'] = pd.Series(
    -np.pmt(
        loanbook['initial_rate']/12,
        loanbook['term'],
        loanbook['loan_amount']
    ),
    index = loanbook.index
)


Personal notes: np.pmt computes the payment against loan principal plus interest

In [13]:
#For monthly_repay_reversion, reversion_balance must first be calculated. Reversion_balance will then replace loan_amount in the np.pmt calculation
def remaining_amount(rate, term, loan_amount, period):
    '''
    The function returns the remaining amount.

    First the principal_paid will be initialized.
    Then the montlhy_repay amount will be calculated
    The loan amount will then be subtracted.
    The cumulative amount paid will be done through iteration. Note that the data has months start at month 0, so range is period - 1

    Returns amount paid 
    '''
    principal_paid = 0 
    
    monthly_repay = -np.pmt(rate, term, loan_amount, 0, 1)

    loan_amount = loan_amount - monthly_repay

    for i in range(period - 1): 
        month_interest = loan_amount * rate
        paid_off = monthly_repay - month_interest
        loan_amount = loan_amount - paid_off

    return loan_amount 

In [14]:
#Now the goal is to add th reversion balance (leftover loan_amount at the reversion date)

#initializing for the next loop
loanbook['reversion_balance'] = 0 
loanbook['monthly_repay_reversion'] = 0

#first calculate the leftover loan amount on reversion date and add it to the reversion_balance columns
for i in range(len(loanbook)):
    loanbook['reversion_balance'].iloc[i] = remaining_amount(
        loanbook['initial_rate'].iloc[i]/12,
        loanbook['term'].iloc[i],
        loanbook['loan_amount'].iloc[i],
        loanbook['rate_term'].iloc[i]*12
    )


In [17]:
#now calculate the monthly repay amount after the reversion date
loanbook['monthly_repay_reversion'] = pd.Series(
    -np.pmt(
        loanbook['reversion_rate']/12,
        loanbook['term'] - loanbook['rate_term'] * 12,
        loanbook['reversion_balance']
    ),
    index = loanbook.index
)